![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
import time
from sklearn.preprocessing import StandardScaler



filename_features = "/content/features.txt"
filename_labels = "/content/activity_labels.txt"

filename_subtrain = "/content/subject_train.txt"
filename_xtrain = "/content/X_train.txt"
filename_ytrain = "/content/y_train.txt"

filename_subtest = "/content/subject_test.txt"
filename_xtest = "/content/X_test.txt"
filename_ytest = "/content/y_test.txt"

# Carregar features e labels
features = pd.read_csv(filename_features, sep='\s+', header=None, names=['idx', 'nome_var'], encoding="utf-8")
features = features['nome_var'].tolist()

# Remover duplicatas dos nomes das variáveis
features = list(dict.fromkeys(features))  # Remove duplicatas mantendo a ordem

labels = pd.read_csv(filename_labels, sep='\s+', header=None, names=['cod_label', 'label'], encoding="utf-8")

# Carregar dados de treino
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'], encoding="utf-8")['subject_id']
X_train = pd.read_csv(filename_xtrain, sep='\s+', header=None, names=features, encoding="utf-8")
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'], encoding="utf-8")

# Carregar dados de teste
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'], encoding="utf-8")['subject_id']
X_test = pd.read_csv(filename_xtest, sep='\s+', header=None, names=features, encoding="utf-8")
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'], encoding="utf-8")

## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [7]:
%%time

# Função de padronização personalizada
def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean()) / s.std()
    return s

# Aplicando a padronização personalizada nos dados de treinamento
X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)

# PCA sem padronização (dados originais)
pca_no_scaling = PCA()
X_pca_no_scaling = pca_no_scaling.fit_transform(X_train)

# PCA com padronização personalizada
pca_with_custom_scaling = PCA()
X_pca_with_custom_scaling = pca_with_custom_scaling.fit_transform(X_train_pad)

# Variância explicada e acumulada - sem padronização
explained_variance_no_scaling = pca_no_scaling.explained_variance_ratio_
cumulative_variance_no_scaling = explained_variance_no_scaling.cumsum()

# Variância explicada e acumulada - com padronização personalizada
explained_variance_with_custom_scaling = pca_with_custom_scaling.explained_variance_ratio_
cumulative_variance_with_custom_scaling = explained_variance_with_custom_scaling.cumsum()

# Exibir os resultados em um DataFrame para comparação
import pandas as pd

resultados = pd.DataFrame({
    "Componente": range(1, len(explained_variance_no_scaling) + 1),
    "Var. Explicada Sem Padronização": explained_variance_no_scaling,
    "Var. Explicada Acumulada Sem Padronização": cumulative_variance_no_scaling,
    "Var. Explicada Com Padronização Customizada": explained_variance_with_custom_scaling,
    "Var. Explicada Acumulada Com Padronização Customizada": cumulative_variance_with_custom_scaling
})

print(resultados)

# Identificar o número de componentes necessários para explicar 90% da variância
num_components_no_scaling = (cumulative_variance_no_scaling >= 0.90).argmax() + 1
num_components_with_custom_scaling = (cumulative_variance_with_custom_scaling >= 0.90).argmax() + 1

print(f"Número de componentes sem padronização para explicar 90% da variância: {num_components_no_scaling}")
print(f"Número de componentes com padronização personalizada para explicar 90% da variância: {num_components_with_custom_scaling}")

     Componente  Var. Explicada Sem Padronização  \
0             1                         0.679477   
1             2                         0.049314   
2             3                         0.021167   
3             4                         0.017638   
4             5                         0.013143   
..          ...                              ...   
472         473                         0.000000   
473         474                         0.000000   
474         475                         0.000000   
475         476                         0.000000   
476         477                         0.000000   

     Var. Explicada Acumulada Sem Padronização  \
0                                     0.679477   
1                                     0.728792   
2                                     0.749959   
3                                     0.767596   
4                                     0.780740   
..                                         ...   
472                      

## Análise de Componentes em Markdown

### Distribuição e Comportamento dos Dados
- **Var. Explicada Sem Padronização:**
  - A maior parte da variância é explicada pelo primeiro componente (**67,95%**), com uma queda significativa nos componentes subsequentes.
  - Após os primeiros 5 componentes, a contribuição para a variância se torna bastante pequena.

- **Var. Explicada Acumulada Sem Padronização:**
  - Os primeiros **29 componentes** explicam **90%** da variância.
  - Sem padronização, poucos componentes são necessários para capturar a maioria da variância dos dados.

---

### Impacto da Padronização Customizada
- **Var. Explicada Com Padronização Customizada:**
  - O primeiro componente explica **54,58%** da variância, seguido por uma redução gradual nos componentes subsequentes.
  - Isso indica que a padronização ajustou os pesos das variáveis explicativas.

- **Var. Explicada Acumulada Com Padronização Customizada:**
  - São necessários **54 componentes** para alcançar **90%** da variância acumulada.
  - Com padronização, mais componentes são necessários para capturar a variância dos dados.

---

### Resumo Comparativo
| Método                     | Componentes para 90% da Variância | Contribuição Inicial (1º Componente) |
|----------------------------|-----------------------------------|--------------------------------------|
| **Sem Padronização**       | 29                               | 67,95%                              |
| **Com Padronização**       | 54                               | 54,58%                              |

---

### Desempenho Computacional
- **CPU Times:** Usuário: **2,3s** | Sistema: **48,3ms** | Total: **2,35s**
- **Wall Time:** **3,44s**

---


## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

In [9]:
%%time

# Função de padronização personalizada
def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean()) / s.std()
    return s

# Aplicar a padronização personalizada nos dados de treinamento
X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)

# Aplicar a padronização personalizada nos dados de teste
X_test_pad = pd.DataFrame(X_test).apply(padroniza, axis=0)

# Reduzir dimensionalidade para 10 componentes principais - sem padronização
pca_no_scaling = PCA(n_components=10)
X_train_pca_no_scaling = pca_no_scaling.fit_transform(X_train)
X_test_pca_no_scaling = pca_no_scaling.transform(X_test)

# Reduzir dimensionalidade para 10 componentes principais - com padronização personalizada
pca_with_scaling = PCA(n_components=10)
X_train_pca_with_scaling = pca_with_scaling.fit_transform(X_train_pad)
X_test_pca_with_scaling = pca_with_scaling.transform(X_test_pad)

# Treinar a árvore de decisão - sem padronização
tree_no_scaling = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
tree_no_scaling.fit(X_train_pca_no_scaling, y_train)
y_pred_no_scaling = tree_no_scaling.predict(X_test_pca_no_scaling)

# Treinar a árvore de decisão - com padronização
tree_with_scaling = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
tree_with_scaling.fit(X_train_pca_with_scaling, y_train)
y_pred_with_scaling = tree_with_scaling.predict(X_test_pca_with_scaling)

# Avaliar desempenho
accuracy_no_scaling = accuracy_score(y_test, y_pred_no_scaling)
accuracy_with_scaling = accuracy_score(y_test, y_pred_with_scaling)

print(f"Acurácia sem padronização: {accuracy_no_scaling}")
print(f"Acurácia com padronização personalizada: {accuracy_with_scaling}")

Acurácia sem padronização: 0.7848659653885307
Acurácia com padronização personalizada: 0.6308109942314218
CPU times: user 1.64 s, sys: 11.2 ms, total: 1.65 s
Wall time: 1.46 s


In [10]:
# Avaliar a acurácia na base de treino - sem padronização
accuracy_train_no_scaling = accuracy_score(y_train, tree_no_scaling.predict(X_train_pca_no_scaling))

# Avaliar a acurácia na base de treino - com padronização personalizada
accuracy_train_with_scaling = accuracy_score(y_train, tree_with_scaling.predict(X_train_pca_with_scaling))

# Exibir os resultados de treino e teste
print(f"Acurácia na base de treino (sem padronização): {accuracy_train_no_scaling}")
print(f"Acurácia na base de teste (sem padronização): {accuracy_no_scaling}")
print(f"Acurácia na base de treino (com padronização personalizada): {accuracy_train_with_scaling}")
print(f"Acurácia na base de teste (com padronização personalizada): {accuracy_with_scaling}")

Acurácia na base de treino (sem padronização): 0.8454842219804135
Acurácia na base de teste (sem padronização): 0.7848659653885307
Acurácia na base de treino (com padronização personalizada): 0.7383025027203483
Acurácia na base de teste (com padronização personalizada): 0.6308109942314218


---

## Análise e Comparação das Acurácias

Ao comparar as acurácias sem padronização e com padronização personalizada, observamos que a aplicação de padronização personalizada teve um impacto negativo nos resultados, tanto na base de treino quanto na base de teste.

### **Resultados sem padronização**
- Na **base de treino**, a acurácia foi de **0.845**. Esse valor indica um bom desempenho do modelo ao aprender os padrões dos dados durante o treinamento.
- Na **base de teste**, a acurácia foi de **0.784**, indicando que o modelo conseguiu generalizar relativamente bem para dados novos.

### **Resultados com padronização personalizada**
- Na **base de treino**, a acurácia caiu para **0.738**. Esse declínio pode significar que a padronização personalizada interferiu na capacidade do modelo de capturar os padrões presentes nos dados de treinamento.
- Na **base de teste**, a acurácia foi de **0.630**, uma queda considerável em comparação aos resultados sem padronização. Isso sugere que a padronização personalizada prejudicou ainda mais a capacidade de generalização do modelo.

### **Comparação**
A padronização personalizada parece ter introduzido um nível de transformação nos dados que dificultou o aprendizado do modelo e sua habilidade de generalização. Apesar de técnicas de padronização frequentemente ajudarem a melhorar o desempenho do modelo, neste caso, os resultados indicam que a abordagem personalizada utilizada não foi eficaz.

---
